# NLP基礎 (1): テキスト前処理、Bag-of-Words (BoW)、TF-IDF

このノートブックでは、自然言語処理 (NLP) の最も基本的なステップであるテキストデータの前処理と、古典的かつ重要なテキスト表現手法であるBag-of-Words (BoW) および TF-IDF (Term Frequency-Inverse Document Frequency) について学びます。
これらの手法は、テキストデータを機械学習アルゴリズムが扱える数値形式に変換するための基礎となります。
主にNumPyを使って、これらの概念をスクラッチで実装し、その動作を理解します。

**参考論文 (TF-IDFの背景として):**
*   Salton, G., & Buckley, C. (1988). Term-weighting approaches in automatic text retrieval. *Information processing & management*, 24(5), 513-523. (この論文は主に情報検索における様々な単語重み付け手法を論じています)

**このノートブックで学ぶこと:**
1.  基本的なテキスト前処理（正規化、トークナイゼーション）。
2.  Bag-of-Words (BoW) モデルの概念と実装。
3.  TF-IDFの概念（TF、IDF）と実装。
4.  これらの手法の長所と短所、そして限界。

**前提知識:**
*   基本的なPythonプログラミングとNumPyの操作。

 ## 1. 必要なライブラリのインポート

In [15]:
import numpy as np
import re # 正規表現によるテキストクリーニング用
from collections import Counter # 単語の頻度カウント用
import math # log計算用

## 2. テキストデータの前処理

機械がテキストデータを理解できるようにするためには、まず生テキストを扱いやすい形に整える「前処理」が必要です。

### 2.1 テキストのクリーニングと正規化

実際のテキストデータには、ノイズとなる文字や、表記の揺れが含まれていることがあります。

*   **小文字化 (Lowercasing):** "Apple" と "apple" を同じ単語として扱うために、全ての文字を小文字（または大文字）に統一します。
*   **句読点・特殊文字の除去:** 文の意味に直接関与しないことが多い句読点（ピリオド、カンマなど）や特殊文字（HTMLタグなど）を除去、または空白に置換します。ただし、文脈によっては重要になる場合もあります（例: 感嘆符が感情を表す）。
*   **数字の扱い:** 数字をそのまま残すか、特別なトークン（例: `<NUM>`）に置き換えるか、あるいは除去するかをタスクに応じて決定します。
*   **Unicode正規化:** 全角文字を半角に統一するなど、文字コードレベルでの正規化。

ここでは、簡単な例として小文字化と、英数字以外の除去を行います。

In [16]:
def preprocess_text_simple(text):
    '''テキストを単純に小文字化し、数字と記号を削除する'''
    text = text.lower()  # 小文字化
    text = re.sub(r'[^a-z0-9\s]', '', text)  # 英小文字，数字，スペース以外を削除
    text = re.sub(r'\s+', ' ', text).strip()  # 連続するスペースを1つにまとめ、前後のスペースを削除
    return text

In [17]:
# テスト
sample_text_raw = "Hello World! This is a Test sentence with numbers 123 and symbols #@$."
processed_text = preprocess_text_simple(sample_text_raw)
print(f"Raw text: '{sample_text_raw}'")
print(f"Processed text: '{processed_text}'")

Raw text: 'Hello World! This is a Test sentence with numbers 123 and symbols #@$.'
Processed text: 'hello world this is a test sentence with numbers 123 and symbols'


### 2.2 トークナイゼーション (Tokenization)

トークナイゼーションは、前処理されたテキストを個々の単語（またはトークン）のリストに分割するプロセスです。
最も単純な方法は、空白文字（スペース、タブ、改行など）で区切ることです。
より高度な方法としては、句読点も考慮したり、言語の文法構造に基づいて分割する形態素解析などがあります。

In [18]:
def tokenize_simple(text):
    '''テキストを単語に分割する'''
    return text.split(' ')

In [19]:
# テスト
tokens = tokenize_simple(processed_text)
print(f"Original processed text: '{processed_text}'")
print(f"Tokens: {tokens}")

# コーパスの準備 (複数の文書からなるリスト)
corpus_raw = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

processed_corpus = [tokenize_simple(preprocess_text_simple(doc)) for doc in corpus_raw]
print("\nProcessed Corpus (list of token lists):")
for i, doc_tokens in enumerate(processed_corpus):
    print(f"Doc {i}: {doc_tokens}")

Original processed text: 'hello world this is a test sentence with numbers 123 and symbols'
Tokens: ['hello', 'world', 'this', 'is', 'a', 'test', 'sentence', 'with', 'numbers', '123', 'and', 'symbols']

Processed Corpus (list of token lists):
Doc 0: ['this', 'is', 'the', 'first', 'document']
Doc 1: ['this', 'document', 'is', 'the', 'second', 'document']
Doc 2: ['and', 'this', 'is', 'the', 'third', 'one']
Doc 3: ['is', 'this', 'the', 'first', 'document']


## 2.3 ステミングとレンマ化

*   **ステミング (Stemming):** 単語を語幹（例: "running" -> "run", "studies" -> "studi"）に変換する処理。単純なルールベースで行われることが多く、必ずしも正しい語幹になるとは限りません。
*   **レンマ化 (Lemmatization):** 単語を見出し語（辞書形、例: "ran" -> "run", "better" -> "good"）に変換する処理。品詞情報などを考慮するため、ステミングより高度で正確ですが、計算コストも高くなります。

これらは語彙のバリエーションを減らし、異なる形の同じ単語を統一的に扱うために行われます。今回のスクラッチ実装では省略しますが、重要な前処理ステップの一つです。

## 3. Bag-of-Words (BoW) モデル

Bag-of-Words (BoW) は、テキストを数値ベクトルとして表現するための最も基本的な手法の一つです。
その名の通り、「単語の袋」として扱い、**文書中での単語の出現順序は無視し、各単語が何回出現したか**という情報のみで文書を表現します。

Salton & Buckley (1988) の論文では、情報検索の文脈で文書やクエリを「term vectors」として表現する考え方が述べられています。
$D = (t_1, t_2, \dots, t_p)$
ここで、$t_k$ は文書 $D$ における $k$番目の単語（またはその重み）を表します。

**BoWベクトルの作成手順:**
1.  **語彙の作成 (Vocabulary Building):**
    コーパス全体（全ての文書）に出現するユニークな単語のリストを作成します。これが語彙（ボキャブラリ）となります。各単語には一意のインデックスが割り当てられます。
2.  **ベクトル化:**
    各文書を、語彙と同じ長さのベクトルに変換します。ベクトルの各要素は、語彙内の対応する単語がその文書中に何回出現したか（単語頻度 - Term Frequency, TF）を表します。

### 3.1 NumPyによるBoWベクトルの作成

In [20]:
def build_vocabulary(tokenized_corpus):
    '''コーパスから語彙を作成し，単語->インデックスのマッピングを返す'''
    all_tokens = [token for doc in tokenized_corpus for token in doc]
    unique_tokens = sorted(list(set(all_tokens)))
    vocab = {token: i for i, token in enumerate(unique_tokens)}
    return vocab

def document_to_bow_vector(doc_tokens, vocab):
    '''1つの文章（トークンのリスト）をBoWベクトルに変換する'''
    vocab_size = len(vocab)
    bow_vector = np.zeros(vocab_size, dtype=np.float32)
    word_counts = Counter(doc_tokens)
    for token, count in word_counts.items():
        if token in vocab:
            bow_vector[vocab[token]] = count
    return bow_vector

In [21]:
# 語彙の作成
vocabulary = build_vocabulary(processed_corpus)
print("Vocabulary (token -> index):\n", vocabulary)
print(f"Vocabulary size: {len(vocabulary)}")

# 各文書をBoWベクトルに変換
bow_vectors = []
for doc_tokens in processed_corpus:
    bow_vectors.append(document_to_bow_vector(doc_tokens, vocabulary))

bow_matrix = np.array(bow_vectors)
print("\nBoW Matrix (documents x vocabulary_size):\n", bow_matrix)
print("BoW Matrix shape:", bow_matrix.shape)

# 語彙の単語リスト (表示用)
vocab_list = [token for token, index in sorted(vocabulary.items(), key=lambda item: item[1])]
print("\nVocabulary List (ordered by index):\n", vocab_list)
print(f"\nBoW for Document 1 ('{ ' '.join(processed_corpus[1]) }'):")
for word, count in zip(vocab_list, bow_matrix[1]):
    if count > 0:
        print(f"  '{word}': {count}")

Vocabulary (token -> index):
 {'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}
Vocabulary size: 9

BoW Matrix (documents x vocabulary_size):
 [[0. 1. 1. 1. 0. 0. 1. 0. 1.]
 [0. 2. 0. 1. 0. 1. 1. 0. 1.]
 [1. 0. 0. 1. 1. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 1. 0. 1.]]
BoW Matrix shape: (4, 9)

Vocabulary List (ordered by index):
 ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

BoW for Document 1 ('this document is the second document'):
  'document': 2.0
  'is': 1.0
  'second': 1.0
  'the': 1.0
  'this': 1.0


### 3.2 BoWの長所と短所

*   **長所:**
    *   シンプルで実装が容易。
    *   テキストのトピックをある程度捉えることができる。
*   **短所:**
    *   **語順の無視:** 単語の出現順序が完全に失われるため、文法的な構造や文脈情報が考慮されない（例: "A B" と "B A" が同じ表現になる）。
    *   **意味の曖昧性:** 同じ単語でも文脈によって意味が異なる場合（多義性）に対応できない。
    *   **次元の爆発:** 語彙サイズが非常に大きくなると、BoWベクトルも高次元になり、スパース（ほとんどの要素が0）になりがち。
    *   **頻出単語の重み:** "the", "is", "a" のような頻出するが重要度の低い単語の重みが大きくなりやすい。

## 4. TF-IDF (Term Frequency-Inverse Document Frequency)

TF-IDFは、BoWの欠点の一つである「頻出単語の重み」の問題を軽減するための手法です。
各単語の重みを、その単語が文書内でどれだけ重要か（**TF**）と、コレクション全体でどれだけ珍しいか（**IDF**）の組み合わせで決定します。

### 4.1 Term Frequency (TF)

TFは、ある単語 $t$ が特定の文書 $d$ 内で出現する頻度です。
最も単純なTFは生の出現回数 $tf(t,d)$ です。
正規化されたTFもよく使われます（例: 文書内の総単語数で割る、対数を取るなど）。
ここでは、簡単のため生の出現回数を使用します（これはBoWベクトルで既に計算済みです）。

### 4.2 Inverse Document Frequency (IDF)

IDFは、ある単語 $t$ がコレクション全体 $D$ の中でどれだけ「珍しい」かを示す指標です。
多くの文書に出現する一般的な単語ほどIDFは低く（重要度が低い）、特定の文書に集中して出現する珍しい単語ほどIDFは高くなります。

計算式:
$idf(t, D) = \log \left( \frac{N}{df_t + 1} \right)$  
*   $N$: コレクション中の総文書数。
*   $df_t$: 単語 $t$ を含む文書の数 (Document Frequency)。
*   分母に `+1` を加えるのは、ある単語がどの文書にも出現しない場合にゼロ除算を防ぐためと、平滑化のためです。（対数の底はeでも10でも2でも良いですが、結果のスケールが変わるだけです。ここでは自然対数 `np.log` を使います。）


### 4.3 TF-IDFスコア

TF-IDFスコアは、TFとIDFの積で計算されます。
$tfidf(t, d, D) = tf(t, d) \times idf(t, D)$
これにより、特定の文書で頻繁に出現し、かつコレクション全体では珍しい単語ほど高いスコアが与えられます。

### 4.4 NumPyによるTF-IDFベクトルの作成

In [22]:
def calculate_tf(bow_matrix):
    '''BoWベクトルからTFを計算する'''
    # 今回は正規化しないのでそのまま返す
    return bow_matrix

def calculate_idf(tokenized_corpus, vocab):
    '''IDFを計算する'''
    num_docs = len(tokenized_corpus)
    vocab_size = len(vocab)
    idf_vector = np.zeros(vocab_size, dtype=np.float32)

    # 各単語がいくつの文章に出現するかをカウント
    doc_counts_per_term = np.zeros(vocab_size, dtype=np.float32)
    for doc_tokens in tokenized_corpus:
        unique_tokens = set(doc_tokens)
        for token in unique_tokens:
            if token in vocab:
                doc_counts_per_term[vocab[token]] += 1

    # IDFの計算
    idf_vector = np.log((num_docs)/(doc_counts_per_term+1))

    return idf_vector

def calculate_tfidf(tf_matrix, idf_vector):
    '''TF-IDFを計算する'''
    return tf_matrix * idf_vector

In [23]:
# 1. TFの計算 (ここではBoWの生の頻度をそのまま使う)
tf_matrix_calc = calculate_tf(bow_matrix) 
# bow_matrix は (num_documents, vocab_size)

# 2. IDFの計算
idf_vector_calc = calculate_idf(processed_corpus, vocabulary)
print("\nIDF Vector:\n", idf_vector_calc)
print(f"IDF for '{vocab_list[vocabulary['document']]}': {idf_vector_calc[vocabulary['document']]:.2f}") # 'document'は3文書に出現
print(f"IDF for '{vocab_list[vocabulary['first']]}': {idf_vector_calc[vocabulary['first']]:.2f}")     # 'first'は2文書に出現
print(f"IDF for '{vocab_list[vocabulary['one']]}': {idf_vector_calc[vocabulary['one']]:.2f}")       # 'one'は1文書にのみ出現 (IDF高)

# 3. TF-IDFの計算
tfidf_matrix_calc = calculate_tfidf(tf_matrix_calc, idf_vector_calc)
print("\nTF-IDF Matrix:\n", tfidf_matrix_calc)

# Document 0 のTF-IDFを確認
print(f"\nTF-IDF for Document 0 ('{ ' '.join(processed_corpus[0]) }'):")
for word, tfidf_score in zip(vocab_list, tfidf_matrix_calc[0]):
    if bow_matrix[0, vocabulary[word]] > 0: # 元のBoWで出現した単語のみ表示
        print(f"  '{word}': TF={bow_matrix[0, vocabulary[word]]}, IDF={idf_vector_calc[vocabulary[word]]:.2f}, TF-IDF={tfidf_score:.2f}")


IDF Vector:
 [ 0.6931472   0.          0.28768212 -0.22314353  0.6931472   0.6931472
 -0.22314353  0.6931472  -0.22314353]
IDF for 'document': 0.00
IDF for 'first': 0.29
IDF for 'one': 0.69

TF-IDF Matrix:
 [[ 0.          0.          0.28768212 -0.22314353  0.          0.
  -0.22314353  0.         -0.22314353]
 [ 0.          0.          0.         -0.22314353  0.          0.6931472
  -0.22314353  0.         -0.22314353]
 [ 0.6931472   0.          0.         -0.22314353  0.6931472   0.
  -0.22314353  0.6931472  -0.22314353]
 [ 0.          0.          0.28768212 -0.22314353  0.          0.
  -0.22314353  0.         -0.22314353]]

TF-IDF for Document 0 ('this is the first document'):
  'document': TF=1.0, IDF=0.00, TF-IDF=0.00
  'first': TF=1.0, IDF=0.29, TF-IDF=0.29
  'is': TF=1.0, IDF=-0.22, TF-IDF=-0.22
  'the': TF=1.0, IDF=-0.22, TF-IDF=-0.22
  'this': TF=1.0, IDF=-0.22, TF-IDF=-0.22


### 4.5 TF-IDFの利点
*   **重要語の重み付け:** 単に頻出するだけでなく、その文書を特徴づけるような重要な単語（特定の文書にはよく出るが、他の文書にはあまり出ない単語）に高い重みを与えることができます。
*   **ストップワードの抑制:** "the", "is", "a" のような多くの文書に共通して出現する単語（ストップワード）は、IDFが低くなるため、TF-IDFスコアも自然と低くなり、その影響を抑制できます。

## 5. 簡単な応用例 (概念)

作成されたBoWベクトルやTF-IDFベクトルは、以下のようなタスクに利用できます。

*   **文書類似度計算:**
    2つの文書ベクトル間のコサイン類似度などを計算することで、内容がどれだけ似ているかを測ることができます。  
    $\text{cosine\_similarity}( \vec{A}, \vec{B} ) = \frac{ \vec{A} \cdot \vec{B} }{ \| \vec{A} \| \| \vec{B} \| }$  
*   **情報検索:**
    クエリも同様にベクトル化し、クエリベクトルと各文書ベクトルとの類似度を計算して、関連性の高い文書をランキングします。
*   **テキスト分類:**
    各文書ベクトルを特徴量として、ロジスティック回帰、SVM、ナイーブベイズなどの機械学習分類器を学習させ、新しい文書のカテゴリを予測します。

これらの応用は、このノートブックの範囲を超えますが、BoWやTF-IDFがどのように使われるかのイメージを持つことは重要です。

## 6. 考察: BoWとTF-IDFの限界と次へのステップ

BoWとTF-IDFは、テキストを数値ベクトル化するための古典的で強力な手法ですが、いくつかの本質的な限界も抱えています。

*   **語順の完全な無視:** 文法的な構造や単語間の関係性（例: "AがBをCする" と "BがAをCする" の違い）を捉えることができません。
*   **意味の表現の限界:**
    *   **同義語:** "buy" と "purchase" のように意味が似ている単語も、異なるトークンとして扱われるため、ベクトル空間上では無関係な次元として表現されてしまいます。
    *   **多義語:** 同じ単語でも文脈によって意味が異なる場合（例: "bank" が銀行か土手か）を区別できません。
*   **高次元性とスパース性:** 語彙サイズが大きくなると、ベクトルは非常に高次元かつスパースになり、計算効率やモデルの性能に影響を与えることがあります。

これらの限界を克服し、単語の「意味」をより豊かに、かつ低次元の密なベクトルで表現しようとする試みが、次の段階で学ぶ**単語埋め込み (Word Embeddings)**、特にword2vecやGloVeといった手法に繋がっていきます。これらの手法は、単語の分散表現（Distributed Representation）を獲得し、意味的に近い単語がベクトル空間上でも近くに配置されるように学習します。